In [1]:
import numpy as np
import h5py
import os
import matplotlib.pyplot as plt
import pandas as pd
import time
# Own module
from mlp import classification_accuracy, MLP

## Data Loading and cleaning

In [2]:
with h5py.File('../Inputs/train_128.h5','r') as H:
    data = np.copy(H['data'])
with h5py.File('../Inputs/train_label.h5', 'r') as H:
    label = np.copy(H['label'])
with h5py.File('../Inputs/test_128.h5','r') as H:
    x_test = np.copy(H['data'])

### Shuffling

In [3]:
def shuffle(x,y):
        """
        shuffles given input and target variables of same first axis shape
        :returns x,y: shuffled input and target
        """
        idxs = list(range(x.shape[0]))
        np.random.shuffle(idxs)
        return x[idxs], y[idxs]

In [4]:
x_train, y_train = shuffle(data, label)

### Feature Engineering

In [5]:
mu_train = x_train.mean()
sigma_train = x_train.std()

mu_test = x_test.mean()
sigma_test = x_test.std()

x_train = (x_train - mu_train)/sigma_train
x_test = (x_test - mu_test)/sigma_test

In [6]:
def _one_hot(x, k, dtype=np.float32):
    """Create a one-hot encoding of x of size k."""
    return np.array(x[:, None] == np.arange(k), dtype)

In [7]:
y_train = _one_hot(y_train, 10)

In [9]:
print(x_train.shape, y_train.shape)

(60000, 128) (60000, 10)


In [10]:
# Validation same for purposes testing
x_val, y_val = x_train, y_train

## Training the optimal model

In [11]:
final_model = MLP(layers = [128,64,32,16,10],
                  activation = [None,'tanh','tanh','relu','softmax'],
                  batch_norm= [None, False, False, False, False],
                  dropouts= [None, 0, 0, 0, 0])

final_loss, final_train_acc, final_val_acc = final_model.fit(X_train= x_train, Y_train=y_train,
                                                             X_val = x_val, Y_val = y_val,
                                                             epochs = 20,batch_size = 128, learning_rate= 0.01,
                                                             beta1 = 0.5, optimizer = 'adam', verbose= True)

end of epoch 0
training accuracy is 0.8617
validation accuracy is 0.8617
training loss is 0.5067
end of epoch 1
training accuracy is 0.8728
validation accuracy is 0.8728
training loss is 0.3799
end of epoch 2
training accuracy is 0.8803
validation accuracy is 0.8803
training loss is 0.3515
end of epoch 3
training accuracy is 0.8859
validation accuracy is 0.8859
training loss is 0.3328
end of epoch 4
training accuracy is 0.8903
validation accuracy is 0.8903
training loss is 0.3187
end of epoch 5
training accuracy is 0.8941
validation accuracy is 0.8941
training loss is 0.3072
end of epoch 6
training accuracy is 0.8972
validation accuracy is 0.8972
training loss is 0.2975
end of epoch 7
training accuracy is 0.8998
validation accuracy is 0.8998
training loss is 0.2891
end of epoch 8
training accuracy is 0.9022
validation accuracy is 0.9022
training loss is 0.2816
end of epoch 9
training accuracy is 0.9043
validation accuracy is 0.9043
training loss is 0.2749
end of epoch 10
training accur

## Test Predictions and Exporting

In [12]:
test_outputs = final_model.predict(x_test)
test_labels = np.argmax(test_outputs, axis = -1)

In [13]:
# Export them
export_file = h5py.File('../Output/test_labels.h5', 'w')
export_file.create_dataset('label', data = test_labels)

<HDF5 dataset "label": shape (10000,), type "<i8">